In [1]:
from random import randint, seed
import numpy as np

In [21]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = [], []
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

seed(42)
n_samples=2
n_numbers = 3
largest = 10
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)

[[2, 1, 5], [4, 4, 3]] [8, 11]


In [22]:
def to_string(X, y, n_numbers, largest):
    X_max_len = int(n_numbers * np.ceil(np.log10(largest+1)) + n_numbers - 1)
    X_str = []
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(X_max_len - len(strp))]) + strp
        X_str.append(strp)
    y_max_len = int(np.ceil(np.log10(n_numbers * (largest + 1))))
    y_str = []
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(y_max_len - len(strp))]) + strp
        y_str.append(strp)
    return X_str, y_str

In [23]:
Xstr, ystr = to_string(X, y, n_numbers, largest)

In [25]:
ystr

[' 8', '11']

In [5]:
def integer_encode(X, y, alphabet):
    """
    Encode string representation of integer as indices in some alphabet
    """
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = []
    for pattern in X:
        int_enc = [char_to_int[c] for c in pattern]
        Xenc.append(int_enc)
    yenc = []
    for pattern in y:
        int_enc = [char_to_int[c] for c in pattern]
        yenc.append(int_enc)
    return Xenc, yenc

In [6]:
alphabet = tuple("0123456789+ ")
n_chars = len(alphabet)
X, y = integer_encode(Xstr, ystr, alphabet)

In [7]:
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = []
        for i in seq:
            vector = [0 for _ in range(max_int)]
            vector[i] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = []
        for i in seq:
            vec = [0 for _ in range(max_int)]
            vec[i] = 1
            pattern.append(vec)
        yenc.append(pattern)
    return Xenc, yenc

In [8]:
X, y = one_hot_encode(X, y, len(alphabet))

In [9]:
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = []
    for pattern in seq:
        s = []
        for sym in pattern:
            s.append(int_to_char[np.argmax(sym)])
        strings.append(''.join(s))
    return strings

In [10]:
invert(X, alphabet)

['   2+1+5', '   4+4+3']

In [11]:
def prep_data(n_samples, n_numbers, largest, alphabet):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = to_string(X, y, n_numbers, largest)
    X, y = integer_encode(X, y, alphabet)
    X, y = one_hot_encode(X, y, len(alphabet))
    return np.array(X), np.array(y)

In [12]:
def gen_data(n_numbers, largest, alphabet, batch_size=32):
    while True:
        X, y = random_sum_pairs(batch_size, n_numbers, largest)
        X, y = to_string(X, y, n_numbers, largest)
        X, y = integer_encode(X, y, alphabet)
        X, y = one_hot_encode(X, y, len(alphabet))
        yield np.array(X), np.array(y)

In [13]:
n_in_seq_length = int(n_numbers * np.ceil(np.log10(largest+1)) + n_numbers - 1)
n_out_seq_length = int(np.ceil(np.log10(n_numbers * (largest+1))))

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, RepeatVector, Dense

Using TensorFlow backend.
/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [15]:
n_batches = 10
n_epoch = 7
model = Sequential()
model.add(LSTM(10, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(5, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                920       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 2, 10)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 5)              320       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2, 12)             72        
Total params: 1,312
Trainable params: 1,312
Non-trainable params: 0
_________________________________________________________________


In [16]:
hist = model.fit_generator(gen_data(n_numbers, largest, alphabet), epochs=10, steps_per_epoch=1000)

Epoch 1/10
1000/1000 [==============================] - 9s 9ms/step - loss: 1.8179 - acc: 0.3434
Epoch 2/10
1000/1000 [==============================] - 8s 8ms/step - loss: 1.6048 - acc: 0.3682
Epoch 3/10
1000/1000 [==============================] - 8s 8ms/step - loss: 1.5807 - acc: 0.3710
Epoch 4/10
1000/1000 [==============================] - 8s 8ms/step - loss: 1.2721 - acc: 0.5380
Epoch 5/10
1000/1000 [==============================] - 7s 7ms/step - loss: 1.0418 - acc: 0.6468
Epoch 6/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.9157 - acc: 0.7505
Epoch 7/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.7957 - acc: 0.8405
Epoch 8/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.7113 - acc: 0.8816
Epoch 9/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6414 - acc: 0.8997
Epoch 10/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.5862 - acc: 0.9006


In [17]:
n_samples = 32
X, y = prep_data(n_samples, n_numbers, largest, alphabet)

In [18]:
res = model.predict(X)

In [19]:
expected = invert(y, alphabet)
predicted = invert(res, alphabet)

In [20]:
for i in range(n_samples):
    print(f"Expected={expected[i]}, Predicted={predicted[i]}")

Expected=21, Predicted=22
Expected=13, Predicted=13
Expected=26, Predicted=29
Expected=11, Predicted=11
Expected=18, Predicted=18
Expected=17, Predicted=17
Expected=20, Predicted=20
Expected=12, Predicted=12
Expected=12, Predicted=12
Expected=18, Predicted=18
Expected=21, Predicted=22
Expected=14, Predicted=14
Expected=23, Predicted=22
Expected=26, Predicted=29
Expected= 9, Predicted= 9
Expected=20, Predicted=20
Expected=15, Predicted=15
Expected=18, Predicted=18
Expected=12, Predicted=12
Expected=20, Predicted=20
Expected=12, Predicted=12
Expected=23, Predicted=22
Expected=20, Predicted=20
Expected=18, Predicted=18
Expected=21, Predicted=22
Expected=15, Predicted=15
Expected=14, Predicted=14
Expected=19, Predicted=19
Expected=20, Predicted=20
Expected=22, Predicted=22
Expected=10, Predicted=10
Expected=17, Predicted=17
